In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [44]:
# Importo los 3 archivos, utilizo ese encoding ya que me da error, busque en Kaggle en los retos a ver cual era. 
df1 = pd.read_csv("PoliceKillingsUS.csv", encoding='Windows-1252')
df2 = pd.read_csv("MedianHouseholdIncome2015.csv",encoding='Windows-1252')
df3 = pd.read_csv("PercentagePeopleBelowPovertyLevel.csv",encoding="Windows-1252")

# Hago el Merge, ahora df contiene a df1 ,la columna "Median income" de df2 y "poverty rate de df3".
df2_int = df2[["City","Median Income"]]
df3_int = df3[["City","poverty_rate"]]
df = df1.merge(df2_int,left_on="city",right_on="City").merge(df3_int,left_on="city",right_on="City")

# Imprimo el head para inspeccionar un poco las columnas que tengo. 
display(df.head(3))

# Veo que tengo repetida varias veces la columna de city asi q me quito las reptidas y nombro al df "police".
police = df[df.columns.difference(['City_x', 'City_y'])]
display(police.head(3))

,id,name,date,manner_of_death,armed,age,gender,race,city,state,signs_of_mental_illness,threat_level,flee,body_camera,City_x,Median Income,City_y,poverty_rate
0,744,Jonathon Pope,15/08/15,shot,gun,30.0,M,W,Carson City,NV,False,attack,Not fleeing,False,Carson City,47668,Carson City,16.8
1,2008,Henry Simpson,01/11/16,shot,gun,76.0,M,W,Carson City,NV,False,other,Not fleeing,False,Carson City,47668,Carson City,16.8
2,828,Joseph Thompson Johnson-Shanks,14/09/15,shot,gun,25.0,M,B,Princeton,KY,False,attack,Car,False,Princeton,114645,Princeton,6


,Median Income,age,armed,body_camera,city,date,flee,gender,id,manner_of_death,name,poverty_rate,race,signs_of_mental_illness,state,threat_level
0,47668,30.0,gun,False,Carson City,15/08/15,Not fleeing,M,744,shot,Jonathon Pope,16.8,W,False,NV,attack
1,47668,76.0,gun,False,Carson City,01/11/16,Not fleeing,M,2008,shot,Henry Simpson,16.8,W,False,NV,other
2,114645,25.0,gun,False,Princeton,14/09/15,Car,M,828,shot,Joseph Thompson Johnson-Shanks,6,B,False,KY,attack


In [ ]:
# Probemos